In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from src.bertsum import BertSummarizerConfig, BertSummarizer
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset

In [14]:
bertsum = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at eReverter/bert-finetuned-cnn_dailymail were not used when initializing BertModel: ['encoder.linear.weight', 'encoder.linear.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
bertsum

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [18]:
from evaluate_model import preprocess_data

In [19]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [22]:
data = load_dataset('eReverter/cnn_dailymail_extractive')

Using custom data configuration eReverter--cnn_dailymail_extractive-724c7cce7ac202ac


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /home/usuaris/veu/enric.reverter/.cache/huggingface/datasets/eReverter___parquet/eReverter--cnn_dailymail_extractive-724c7cce7ac202ac/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
data

DatasetDict({
    test: Dataset({
        features: ['src', 'tgt', 'labels'],
        num_rows: 11490
    })
    validation: Dataset({
        features: ['src', 'tgt', 'labels'],
        num_rows: 13368
    })
    train: Dataset({
        features: ['src', 'tgt', 'labels'],
        num_rows: 287113
    })
})

In [36]:
import itertools
from copy import copy

def prepare_sample(sample,
                   tokenizer,
                   max_src_ntokens=200, 
                   min_src_ntokens=5,
                   max_nsents=100,
                   max_length=512,
                   return_tensors=True):
    """
    Prepare sample to run inference.
    `sample` is of the form [sentence1, sentence2, ...]
    """
    inputs = {}

    # Prepare the right input for BERT
    src = tokenizer(
        sample,
        max_length=max_src_ntokens,
        truncation=True,
        stride=0,
        return_token_type_ids=False,
        return_attention_mask=False
    )
    
    # Ignore senteces that are too short
    # *Assumption*: if sentence is short it is not relevant
    idxs = [i for i, sentence in enumerate(src['input_ids']) if (len(sentence) > min_src_ntokens)]

    # Trim sentences to a maximum. Note they are already trimmed by the tokenizer
    src = [src['input_ids'][i] for i in idxs]
    sample = [sample[i] for i in idxs]
    src = src[:max_nsents]

    # Flatten into a single sequence (sents will be separated by [SEP] and [CLS] tokens already)
    src = list(itertools.chain(*src))
    if len(src) > max_length:
        src = src[:max_length-1] + [tokenizer.sep_token_id] # Truncate to 512 tokens

    # Intercalate 0s and 1s to differentiate between sentences
    _segs = [-1] + [i for i, t in enumerate(src) if t == tokenizer.sep_token_id]
    segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
    segment_ids = []
    for i, s in enumerate(segs):
        if (i % 2 == 0):
            segment_ids += s * [0]
        else:
            segment_ids += s * [1]

    # Get [CLS] positions, trim labels
    cls_ids = [i for i, t in enumerate(src) if t == tokenizer.cls_token_id]

    # Store data
    del _segs, segs, idxs
    inputs['input_ids'] = src
    inputs['mask'] = [1] * len(src)
    inputs['segment_ids'] = segment_ids
    inputs['cls_ids'] = cls_ids
    inputs['mask_cls'] = [1] * len(cls_ids)
    inputs['sample'] = sample

    return inputs

In [34]:
sample = data['test'][0]['src']
sample

['(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories.',
 'The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based.',
 'The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014."',
 'Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis.',
 'As members of the court, Palestinians may be subject to counter-charges as well.',
 "Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body.",
 "But Palestinian Foreign Minister Riad al-Malki, speaking 

In [9]:
checkpoint = 'eReverter/bert-finetuned-cnn_dailymail'

In [12]:
bertsum = BertSummarizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
model_inputs = prepare_sample(sample, tokenizer)

KeyError: 'sample'